In [1]:
#Import required libraries
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
from geopy import Nominatim

# Scraping a list of world capitals

In [2]:
# Load thd page and get it into so yummy Beautiful Soup
html_doc = 'https://www.boldtuesday.com/pages/alphabetical-list-of-all-countries-and-capitals-shown-on-list-of-countries-poster'
page = requests.get(html_doc)
soup = BeautifulSoup(html_doc, 'html.parser')
soup = BeautifulSoup(page.content, 'html.parser')

/usr/local/lib/python3.7/site-packages/bs4/__init__.py:375: UserWarning: "https://www.boldtuesday.com/pages/alphabetical-list-of-all-countries-and-capitals-shown-on-list-of-countries-poster" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


In [3]:
print(soup)

<!DOCTYPE html>

<!--[if lt IE 7 ]><html class="ie ie6" lang="en"> <![endif]-->
<!--[if IE 7 ]><html class="ie ie7" lang="en"> <![endif]-->
<!--[if IE 8 ]><html class="ie ie8" lang="en"> <![endif]-->
<!--[if IE 9 ]><html class="ie ie9" lang="en"> <![endif]-->
<!--[if (gte IE 10)|!(IE)]><!--><html lang="en"> <!--<![endif]-->
<head>
<meta charset="utf-8"/>
<meta content="on" http-equiv="cleartype"/>
<meta content="index,follow" name="robots"/>
<title>ALPHABETICAL LIST OF WORLD COUNTRIES AND CAPITALS - Bold Tuesday</title>
<script>
  window.Personizely = {
    cart: {"token":"b21094806bb2092ca62fb44b33aee9b3","note":null,"attributes":{},"original_total_price":0,"total_price":0,"total_discount":0,"total_weight":0.0,"item_count":0,"items":[],"requires_shipping":false,"currency":"EUR","items_subtotal_price":0,"cart_level_discount_applications":[]},
    customerId: '',
    moneyFormat: '€{{amount}}​',
    cartCollections:{},
    productCollections:[],
  }; 
  </script>
<link href="https://sta

In [4]:
data = []
table = soup.find('table')
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele])


In [5]:
data

[['COUNTRY', 'CAPITAL'],
 ['AFGHANISTAN', 'KABUL'],
 ['ALBANIA', 'TIRANA'],
 ['ALGERIA', 'ALGIERS'],
 ['ANDORRA', 'ANDORRA LA VELLA'],
 ['ANGOLA', 'LUANDA'],
 ['ANTIGUA & BARBUDA', "SAINT JOHN'S"],
 ['ARGENTINA', 'BUENOS AIRES'],
 ['ARMENIA', 'YEREVAN'],
 ['AUSTRALIA', 'CANBERRA'],
 ['AUSTRIA', 'VIENNA'],
 ['AZERBAIJAN', 'BAKU'],
 ['BAHAMAS, THE', 'NASSAU'],
 ['BAHRAIN', 'MANAMA'],
 ['BANGLADESH', 'DHAKA'],
 ['BARBADOS', 'BRIDGETOWN'],
 ['BELARUS', 'MINSK'],
 ['BELGIUM', 'BRUSSELS'],
 ['BELIZE', 'BELMOPAN'],
 ['BENIN', 'PORTO-NOVO'],
 ['BHUTAN', 'THIMPHU'],
 ['BOLIVIA', 'SUCRE'],
 ['BOSNIA & HERZEGOVINA', 'SARAJEVO'],
 ['BOTSWANA', 'GABORONE'],
 ['BRAZIL', 'BRASILIA'],
 ['BRUNEI', 'BANDAR SERI BEGAWAN'],
 ['BULGARIA', 'SOFIA'],
 ['BURKINA FASO', 'OUAGADOUGOU'],
 ['BURUNDI', 'BUJUMBURA'],
 ['CABO VERDE', 'PRAIA'],
 ['CAMBODIA', 'PHNOM PENH'],
 ['CAMEROON', 'YAOUNDE'],
 ['CANADA', 'OTTAWA'],
 ['CENTRAL AFRICAN REPUBLIC', 'BANGUI'],
 ['CHAD', "N'DJAMENA"],
 ['CHILE', 'SANTIAGO'],
 ['CHINA

In [6]:
cities = []

for el in data[1:]:
    cities.append(el[1])

In [7]:
cities_df = pd.DataFrame(cities)

In [8]:
cities_df

,0
0,KABUL
1,TIRANA
2,ALGIERS
3,ANDORRA LA VELLA
4,LUANDA
...,...
191,CARACAS
192,HANOI
193,SANA'A
194,LUSAKA


In [9]:
cities_df.rename(columns = {0:'City'}, inplace = True)

In [10]:
len(cities_df)

196

In [11]:
cities_df.isnull().any()

City    False
dtype: bool

In [12]:
# the length of dataframe is quite big, so I will randomly pick 100 cities
# divide into two:

sample = cities_df.sample(n = 100)

# Appending coordinates

In [14]:
# creating a geolocator to use it to get coordinates, increasing timeout.
geolocator = Nominatim(timeout = 5)

# applying the geolocator
sample['Latitude'] = sample['City'].apply(geolocator.geocode).apply(lambda x: x.latitude)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  


In [15]:
sample['Longitude'] = sample['City'].apply(geolocator.geocode).apply(lambda x: x.longitude)